<a href="https://colab.research.google.com/github/SarthakNarayan/DL-and-ML/blob/master/googlecolab/KaggleCatsAndDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mouting Google drive

In [0]:
# mounting google drive for saving the weights
from google.colab import drive
drive.mount('/content/gdrive')

#Getting the data from kaggle

In [0]:
# importing the json file
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
# commands for loading kaggle dataset
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# loading the bees and ants dataset from kaggle
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 30.3MB/s]
 98% 266M/271M [00:01<00:00, 211MB/s]
100% 271M/271M [00:01<00:00, 206MB/s]
 99% 537M/543M [00:04<00:00, 153MB/s]
100% 543M/543M [00:04<00:00, 127MB/s]


In [0]:
# Unzipping the data
!unzip "/content/test1.zip"
!unzip "/content/train.zip"

In [0]:
# data has to be arranged in folders for training and test data to be used by pytorch
!mkdir -p /content/trainArranged/cats
!mkdir -p /content/trainArranged/dogs
!mkdir -p /content/testArranged/cats
!mkdir -p /content/testArranged/dogs

In [0]:
import os
path_from_train = "/content/train"
path_to_train = "/content/trainArranged"

train_contents = os.listdir(path_from_train)
test_contents = os.listdir(path_from_test)

for files in train_contents:
    if(files[0] == 'c'): 
        os.rename(path_from_train+"/"+files , "/content/trainArranged/cats/"+files)
    else:
        os.rename(path_from_train+"/"+files , "/content/trainArranged/dogs/"+files)

# cats is label 0 and dogs is label 1 

#ETL Pipeline

In [15]:
# Loading the train and test data
import torch
import torchvision
from torchvision import transforms

# data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

trainset = torchvision.datasets.ImageFolder(root='/content/trainArranged', 
                                            transform=data_transforms['train'],
                                            )

print("No of images in training set {}".format(len(trainset)))
# since in kaggle competitions test sets are not labelled we will split our train set from 
# training set

No of images in training set 25000


In [17]:
# creating a validation and training set
len_validationset = int(0.1*len(trainset))
len_testset = int(0.1*len(trainset))
print("The length of validation and test set is {}".format(len_validationset))

new_length_trainset = len(trainset) - len_validationset - len_testset
trainset , validationset , testset = torch.utils.data.random_split(trainset , 
                                                                   [new_length_trainset,
                                                                     len_validationset , 
                                                                    len_testset])

# creating the data loaders
train_loader = torch.utils.data.DataLoader(trainset, batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100,shuffle=True)
validation_loader = torch.utils.data.DataLoader(validationset, batch_size=100,shuffle=True)

print("No of batches in train loader {}".format(len(train_loader)))
print("No of batches in test loader {}".format(len(test_loader)))
print("No of batches in validation loader {}".format(len(validation_loader)))

The length of validation and test set is 2000
No of batches in train loader 160
No of batches in test loader 20
No of batches in validation loader 20


#Loading/Making the Model

In [0]:
# Since we are performing transfer learning we will load a model
from torchvision import models
# loading a resnet model with 18 layers
transfer_model = models.resnet18(pretrained=True)
print(transfer_model)
# on visualizing the layer we observe we have to replace the fc layer since we have only 2 classes

In [0]:
import torch.nn.functional as F
import torch.nn as nn

class ReplacementLayer(nn.Module):
    def __init__(self):
        super(ReplacementLayer , self).__init__()
        self.fc1 = nn.Linear(512 , 128)
        self.fc2 = nn.Linear(128 , 64)
        self.output = nn.Linear(64 , 2)
        
    def forward(self , x):
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.output(x)
        return x

end_layer = ReplacementLayer()
print("The last layers are :")
print(end_layer)

In [0]:
# attaching our end_layer inplace of resnet fc layerer
transfer_model.fc = end_layer
# converting the model to make it cuda compatible
transfer_model = transfer_model.cuda()
print(transfer_model)

#Training Process

In [22]:
import torch.optim as optim
# loss function
criterion = nn.CrossEntropyLoss()
# defining the optimizer
# Since we want other layers to remain frozen and only update the weights of our fc layer
optimizer = optim.Adam(transfer_model.fc.parameters() , lr=1e-3)

num_epochs = 5
accuracy_max = 0
for i in range(num_epochs):
    
    running_train_loss = 0
    correct = 0
    accuracy = 0
    running_validation_loss = 0
    
    for images , labels in train_loader:
        # making images and labels cuda compatible
        images = images.cuda()
        labels = labels.cuda()
        # making the gradients zero
        optimizer.zero_grad()
        #forward pass
        logits = transfer_model(images)
        # calculating the loss
        loss = criterion(logits , labels)
        # backward propagation
        loss.backward()
        optimizer.step()
        running_train_loss += loss
    
    print("Training loss after epoch {}/{} is {} ".format(i+1 , num_epochs , running_train_loss/len(train_loader)))
    
    # putting the mode in evaluation mode
    transfer_model.eval()
    
    # Since we dont want to compute gradients 
    with torch.no_grad():
        for images_val , labels_val in validation_loader:
            images_val = images_val.cuda()
            labels_val = labels_val.cuda()
            
            prediction = transfer_model(images_val)
            values , indices = torch.max(prediction , 1)
            valid_loss = criterion(prediction , labels_val)
            
            running_validation_loss += valid_loss
            for j in range(len(indices)):
                if (indices[j] == labels_val[j]):
                    correct += 1
    
    accuracy = (correct/len(validationset))*100
    running_validation_loss = running_validation_loss/len(validation_loader)
    print("Validation loss and accuracy after epoch {}/{} is {} and {}".format(i+1 , 
                                                                               num_epochs , 
                                                                               running_validation_loss,
                                                                               accuracy))
    transfer_model.train()
    if accuracy_max < accuracy:
        accuracy_max = accuracy
        print("Maximum validation accuracy of {} at epoch {}/{}".format(accuracy_max,
                                                                    i+1 , 
                                                                    num_epochs))
        print("saving the model \n")
        torch.save(transfer_model.state_dict(), '/content/gdrive/My Drive/Colab Notebooks/TransferLearning.pth')
    else:
        print()

print("\n Training Over")

Training loss after epoch 1/5 is 0.1738729029893875 
Validation loss and accuracy after epoch 1/5 is 0.13371403515338898 and 94.65
Maximum validation accuracy of 94.65 at epoch 1/5
saving the model 

Training loss after epoch 2/5 is 0.1328398436307907 
Validation loss and accuracy after epoch 2/5 is 0.1652081459760666 and 92.75

Training loss after epoch 3/5 is 0.14476324617862701 
Validation loss and accuracy after epoch 3/5 is 0.1498027890920639 and 93.35

Training loss after epoch 4/5 is 0.12865065038204193 
Validation loss and accuracy after epoch 4/5 is 0.1231127604842186 and 94.95
Maximum validation accuracy of 94.95 at epoch 4/5
saving the model 

Training loss after epoch 5/5 is 0.13060401380062103 
Validation loss and accuracy after epoch 5/5 is 0.1254262775182724 and 94.6


 Training Over


In [0]:
# loading the weights of the best model for testing
model_loaded = transfer_model
model_loaded.cuda()
model_loaded.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/TransferLearning.pth')) 
model_loaded.eval()

In [24]:
# testing the model
correct = 0
accuracy = 0
with torch.no_grad():
        for images_test , labels_test in test_loader:
            images_test = images_test.cuda()
            labels_test = labels_test.cuda()
            prediction = model_loaded(images_test)
            values , indices = torch.max(prediction , 1)
            test_loss = criterion(prediction , labels_test)
            
            for j in range(len(indices)):
                if (indices[j] == labels_test[j]):
                    correct += 1
    
accuracy = (correct/len(testset))*100
print("Test accuracy is {}".format(accuracy))

Test accuracy is 94.95
